In [ ]:
# ! python -m pip install stactools

In [ ]:
import numpy as np
from coverage import *
from datetime import datetime, timezone, timedelta
import dataclasses
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import branca
import folium

from skyfield.framelib import itrs
from landsat import Instrument, Platform, Scene

! mkdir tmp

In [ ]:
start_dt = datetime.fromisoformat(Scene.start_utc)
num_days = 8

tles = gen_sats(
    # sat_nos=[Platform.norad_id] # How to best handle multiple platforms? (TLE vs. SPG4 model too)
    # sat_nos=[39084]
    sat_nos=[39084,49260]
)

inst = camera_model(
    name=Instrument.name, 
    fl=Instrument.focal_length_mm, 
    pitch=Instrument.pitch_um*1e-3, 
    h_pix=Instrument.rows, 
    v_pix=Instrument.cols, 
)

times = gen_times(
    start_yr=start_dt.year,
    start_mo=start_dt.month, 
    start_day=start_dt.day, 
    days=num_days, 
    step_min=Instrument.img_period)

xcell_size = ycell_size = .1

In [ ]:
## Batch FOV generation over N satellites - TODO: build multiple sats into config/ main script
gdfs = []
for tle in tles:
    sat = tle[0]

    fov_df = forecast_fovs(sat, times, inst)

    xyz_dist_rates = sat.at(times).frame_xyz_and_velocity(itrs)
    fov_df['x_pos'], fov_df['y_pos'], fov_df['z_pos'] = xyz_dist_rates[0].km
    fov_df['x_vel'], fov_df['y_vel'], fov_df['z_vel'] = xyz_dist_rates[1].km_per_s

    gdfs.append(fov_df)
fov_df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="EPSG:4326")

# fov_df.drop('datetime', axis=1).explore()

In [ ]:
# fov_df.time = pd.to_datetime(fov_df.time)
fov_df["frac_days"] = fov_df.datetime.dt.dayofweek + fov_df.datetime.dt.hour/24 + fov_df.datetime.dt.minute/(24*60) + fov_df.datetime.dt.second/(24*60*60)
fov_df['time_gap'] = fov_df['frac_days'] - fov_df['frac_days'].shift(1)
# fov_df = fov_df.drop('datetime', axis=1)
fov_df

In [ ]:
# %%timeit
# get_inst_fov(sat, times[0], inst)

In [ ]:
# fov_df.loc[fov_df.z_vel < 0]['asc_dsc'] = 'dsc'
conditions = [
    fov_df['z_vel'].lt(0),
    fov_df['z_vel'].gt(0)
]

choices = ['dsc','asc']
fov_df['asc_dsc'] = np.select(conditions, choices, default='undefined')

my_column_changes = fov_df["asc_dsc"].shift() != fov_df["asc_dsc"]
change = fov_df[my_column_changes]#[:100]
change['time_gap'] = change['frac_days'].shift(-1) - change['frac_days']

# plot_df = change[change.satellite=="LANDSAT 9"][1:]
# fig, ax = plt.subplots(1, figsize=(16,6))
# ax.barh(plot_df['asc_dsc'], plot_df.time_gap, left=plot_df.frac_days) #, color=df.color)

In [ ]:
## Drop ascending pass FOVs
fov_df.loc[fov_df.asc_dsc=="asc", "geometry"] = None

## Select AOI from gpd naturalearth dataset (filter by .name for country, .continent for continent)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world[world.continent != "Antarctica"]
# world = world[world.continent != "Seven seas (open ocean)"]

## Drop any FOVs not over land area
join = fov_df.sjoin(world, how="left").dropna()
fov_df.loc[join.index, 'mode'] = "SCIENCE"
fov_df.loc[~fov_df.index.isin(join.index), 'mode'] = "STANDBY"
fov_df.loc[fov_df['mode']=="STANDBY", "geometry"] = None

In [ ]:
my_column_changes = fov_df["mode"].shift() != fov_df["mode"]

change = fov_df[my_column_changes]#[:100]

change['time_gap'] = change['frac_days'].shift(-1) - change['frac_days']
change = change[change['time_gap'] > 0]
change = change[change.satellite == "LANDSAT 9"]

fig, ax = plt.subplots(1, figsize=(16,2))
ax.barh(change['mode'], change.time_gap, left=change.frac_days) #, color=df.color)

In [ ]:
((change[change["mode"] == "SCIENCE"]).time_gap*24*60).plot.hist(bins=30)

In [ ]:
change[change["mode"] == "SCIENCE"].time_gap.sum()/(change[change["mode"] == "STANDBY"].time_gap.sum() + change[change["mode"] == "SCIENCE"].time_gap.sum())

In [ ]:
def zero_crossing(data):
    return np.where(np.diff(np.sign(np.array(data))))[0]

def crossings_nonzero_pos2neg(data):
    pos = data > 0
    return (pos[:-1] & ~pos[1:]).nonzero()[0]

single_sat = fov_df[fov_df.satellite=="LANDSAT 8"][1:-1].reset_index(drop=True)

data = single_sat['z_pos'].values
crossings = crossings_nonzero_pos2neg(data)
single_sat.loc[crossings]
single_sat['orbit'] = 1

for crossing in crossings:
    single_sat.loc[crossing:, 'orbit'] += 1

In [ ]:
ax = single_sat.plot.line(y=['z_pos', 'z_vel'], xlim=[1000,3000], figsize=(20,8), subplots=False, secondary_y=['z_vel'])
ax.vlines(single_sat.loc[crossings].index, ymin=-1500, ymax=1500, color=['black'])

In [ ]:
(single_sat[single_sat["mode"] == "SCIENCE"].groupby('orbit').time_gap.sum()*24*60).plot.hist(bins=10)

print((single_sat[single_sat["mode"] == "SCIENCE"].groupby('orbit').time_gap.sum()*24*60).mean())

In [ ]:
single_sat['timedelta'] = single_sat.datetime.diff(1).shift(-1)

by_orbit = pd.DataFrame(
    {
    "SCIENCE": single_sat[single_sat['mode']=="SCIENCE"].groupby('orbit').timedelta.sum(),
    "STANDBY": single_sat[single_sat['mode']=="STANDBY"].groupby('orbit').timedelta.sum()
    }
)

by_orbit["SCIENCE_min"] = by_orbit.SCIENCE.dt.total_seconds()/60
by_orbit["STANDBY_min"] = by_orbit.STANDBY.dt.total_seconds()/60

by_orbit["DATA_LWIR_HS"] = by_orbit.SCIENCE.dt.total_seconds()*0.201
by_orbit["DATA_LWIR_HR"] = by_orbit.SCIENCE.dt.total_seconds()*0.60

In [ ]:
by_orbit

In [ ]:
by_orbit.DATA_LWIR_HR.plot.hist(bins=10)

In [ ]:
by_orbit.DATA_LWIR_HR.sum()

In [ ]:
0.16*24*60*60*0.6